In [1]:
!pip install -U sacrebleu datasets transformers sentencepiece protobuf accelerate evaluate unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import get_linear_schedule_with_warmup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load your dataset with pandas
file_path = '/content/drive/MyDrive/combined_df.csv' # Update with your file path
df = pd.read_csv(file_path)

In [8]:
df.dropna(inplace=True)

In [6]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)

# Assuming `df` is your DataFrame with 'hr' and 'nl' columns

# Split the DataFrame into training and validation DataFrames
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)

# Convert the training and validation DataFrames into Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize tokenizer and model
model_checkpoint = "/content/drive/MyDrive/models-finetuning-1/mbart50_large_hr_nl/finetuned-mbart-large-50-hr-to-nl-mmt/checkpoint-46095"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Model configuration settings
model.config.max_length = 64
model.config.early_stopping = True
model.config.num_beams = 5

# Specify source and target languages
tokenizer.src_lang = "hr_HR"
tokenizer.tgt_lang = "nl_XX"

# Define preprocess function to tokenize inputs and targets
def preprocess_function(examples):
    inputs = examples["hr"]
    targets = examples["nl"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True)

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['hr', 'nl', '__index_level_0__'])
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=['hr', 'nl', '__index_level_0__'])

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments
args = Seq2SeqTrainingArguments(
    output_dir="finetuned-mbart-minimal-effect",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-9,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.1,
    num_train_epochs=3,
    predict_with_generate=False,
    fp16=True,
    logging_dir='./logs',
    logging_steps=200,
    warmup_steps=0,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    label_smoothing_factor=0,
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Now you can start training
# trainer.train()

Map:   0%|          | 0/53280 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2805 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.423800,2.326803
2,2.422100,2.326418
3,2.398300,2.326369


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception i

TrainOutput(global_step=2499, training_loss=2.4027848505124707, metrics={'train_runtime': 476.86, 'train_samples_per_second': 335.193, 'train_steps_per_second': 5.241, 'total_flos': 3862705642930176.0, 'train_loss': 2.4027848505124707, 'epoch': 3.0})

In [11]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')
evaluation_df = pd.read_csv('/content/drive/MyDrive/evaluation_df.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [13]:
def translate_croatian_to_dutch(texts):
    # Set the tokenizer's source language code for Croatian. mBART uses 'hr_XX' for Croatian.
    tokenizer.src_lang = "hr_HR"
    # Encode the Croatian texts
    encoded_hr = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=64).to('cuda')
    generated_tokens = model.generate(**encoded_hr, forced_bos_token_id=tokenizer.lang_code_to_id["nl_XX"])
    # Decode the generated tokens to texts
    return [tokenizer.decode(g, skip_special_tokens=True) for g in generated_tokens]

In [14]:
from tqdm.auto import tqdm
import torch
from sacrebleu.metrics import BLEU

bleu = BLEU()

# Prepare lists for translations and BLEU scores
croatian_to_dutch_translations = []
croatian_to_dutch_bleu_scores = []

batch_size = 64  # Set the batch size for processing
for i in tqdm(range(0, len(evaluation_df), batch_size)):
    batch = evaluation_df.iloc[i:i+batch_size]

    # Translate batches from Croatian to Dutch
    translated_dutch_batch = translate_croatian_to_dutch(batch['Croatian'].tolist())  # Batch translation function

    # Extend the translation list with the translated batch
    croatian_to_dutch_translations.extend(translated_dutch_batch)

    # Compute BLEU scores for Dutch translations in batch
    for translated_dutch, original_dutch in zip(translated_dutch_batch, batch['Dutch'].tolist()):
        dutch_score = bleu.corpus_score([translated_dutch], [[original_dutch]])
        croatian_to_dutch_bleu_scores.append(dutch_score.score)

# Update DataFrame with translated Dutch and BLEU scores
evaluation_df['Translated Dutch'] = croatian_to_dutch_translations
evaluation_df['Croatian to Dutch BLEU'] = croatian_to_dutch_bleu_scores

  0%|          | 0/35 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [15]:
evaluation_df

,Unnamed: 0,Dutch,Croatian,Translated Dutch,Croatian to Dutch BLEU
0,0,Op maandag kondigden wetenschappers van de Sta...,U ponedjeljak su znanstvenici s Medicinskog fa...,Op maandag kondigen wetenschappers van de Stan...,24.882579
1,1,Hoofdonderzoekers zeggen dat dit kan leiden to...,Voditelji istraživanja izjavili su da bi ovo o...,Onderzoeksleiders zeiden dat deze ontdekking v...,20.861583
2,2,De JAS 39C Gripen stortte rond 09.30 uur lokal...,JAS 39C Gripen srušio se na pistu i eksplodira...,JAS 39C Gripen crashte op de pier en explodeer...,13.995006
3,3,De piloot werd geïdentificeerd als majoor Dilo...,Pilot je identificiran kao zapovjednik eskadri...,De piloot werd geïdentificeerd als de commanda...,38.677063
4,4,De lokale media meldt dat er tijdens een actie...,Lokalni mediji izvješćuju da je došlo do prevr...,Lokale media melden dat er een omkeer was van ...,5.237521
...,...,...,...,...,...
2221,212,Mijn naam is Andrea.,Moje ime je Andrea.,Mijn naam is Andrea.,100.000000
2222,213,Jupiter is een planeet.,Jupiter je planet.,Jupiter is een planeet.,100.000000
2223,214,Venus is een planeet.,Venera je planet.,Venus is een planeet.,100.000000
2224,215,God is een olifant.,Bog je slon.,God is een olifant.,100.000000


In [16]:
import pandas as pd
from nltk.translate.meteor_score import single_meteor_score
from nltk import word_tokenize
import nltk
from tqdm import tqdm

# Download NLTK's tokenizer models
nltk.download('punkt')
nltk.download('wordnet')

# Initialize an empty list for the METEOR scores
croatian_to_dutch_meteor_scores = []

# Assuming evaluation_df is your DataFrame containing the 'Croatian' and 'Translated_Dutch' columns
# which are the original Croatian sentences and their Dutch translations, respectively.

# Loop through each row in the DataFrame
for _, row in tqdm(evaluation_df.iterrows(), total=evaluation_df.shape[0]):
    # Tokenize the Croatian and Dutch sentences
    original_croatian = word_tokenize(row['Dutch'])  # Original Croatian sentences
    translated_dutch = word_tokenize(row['Translated Dutch'])  # Translated sentences from Croatian to Dutch

    # Calculate the METEOR score for Croatian to Dutch translation
    meteor_score = single_meteor_score(original_croatian, translated_dutch)
    croatian_to_dutch_meteor_scores.append(meteor_score)

# Add the METEOR scores to your DataFrame
evaluation_df['Croatian to Dutch METEOR'] = croatian_to_dutch_meteor_scores

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
100%|██████████| 2226/2226 [00:04<00:00, 533.77it/s] 


In [17]:
from evaluate import load
from tqdm import tqdm

# Load the COMET metric
comet_metric = load('comet')

# Function to compute COMET scores in batches
def compute_comet_in_batches(sources, predictions, references, batch_size=100):
    scores = []
    for i in tqdm(range(0, len(sources), batch_size)):
        batch_sources = sources[i:i + batch_size]
        batch_predictions = predictions[i:i + batch_size]
        batch_references = references[i:i + batch_size]
        batch_results = comet_metric.compute(predictions=batch_predictions, references=batch_references, sources=batch_sources)
        scores.extend(batch_results["scores"])
    return scores

# Assuming evaluation_df is your DataFrame with the necessary columns

# Croatian to Dutch COMET scores
data_cro_to_dut = {
    "sources": evaluation_df['Croatian'].tolist(),  # Original Croatian sentences
    "predictions": evaluation_df['Translated Dutch'].tolist(),  # Translated sentences in Dutch
    "references": evaluation_df['Dutch'].tolist()  # Reference sentences in Dutch
}

# Calculate the COMET scores
cro_to_dut_scores = compute_comet_in_batches(data_cro_to_dut['sources'], data_cro_to_dut['predictions'], data_cro_to_dut['references'])
evaluation_df['Croatian to Dutch COMET'] = [round(score, 5) for score in cro_to_dut_scores]

# Display the updated DataFrame
print(evaluation_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  0%|          | 0/23 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:L

   Unnamed: 0                                              Dutch  \
0           0  Op maandag kondigden wetenschappers van de Sta...   
1           1  Hoofdonderzoekers zeggen dat dit kan leiden to...   
2           2  De JAS 39C Gripen stortte rond 09.30 uur lokal...   
3           3  De piloot werd geïdentificeerd als majoor Dilo...   
4           4  De lokale media meldt dat er tijdens een actie...   

                                            Croatian  \
0  U ponedjeljak su znanstvenici s Medicinskog fa...   
1  Voditelji istraživanja izjavili su da bi ovo o...   
2  JAS 39C Gripen srušio se na pistu i eksplodira...   
3  Pilot je identificiran kao zapovjednik eskadri...   
4  Lokalni mediji izvješćuju da je došlo do prevr...   

                                    Translated Dutch  Croatian to Dutch BLEU  \
0  Op maandag kondigen wetenschappers van de Stan...               24.882579   
1  Onderzoeksleiders zeiden dat deze ontdekking v...               20.861583   
2  JAS 39C Gri

In [18]:
# Calculate average scores for each metric
average_croatian_to_dutch_bleu = evaluation_df['Croatian to Dutch BLEU'].mean()
average_croatian_to_dutch_meteor = evaluation_df['Croatian to Dutch METEOR'].mean()
average_croatian_to_dutch_comet = evaluation_df['Croatian to Dutch COMET'].mean()

# Print the averages
print("Translation scores from Croatian to Dutch:")
print(f"Average BLEU: {average_croatian_to_dutch_bleu}")
print(f"Average METEOR: {average_croatian_to_dutch_meteor}")
print(f"Average COMET: {average_croatian_to_dutch_comet}")

Translation scores from Croatian to Dutch:
Average BLEU: 20.330971995304136
Average METEOR: 0.47615484516768486
Average COMET: 0.8357020215633423


In [19]:
evaluation_df.to_csv('/content/drive/MyDrive/evaluation_df_mbart50_hr_nl.csv')

Isus Krist mir i ljubav